In [1]:
import pandas as pd
import sqlite3

In [ ]:
c = sqlite3.connect("Crime.db")

In [5]:
#csv from https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2
DF = pd.read_csv('AllCrimes.csv')
DF.index = DF.iloc[:,0]

In [6]:
#remove weapons violation, public peace violation, public indecency, offense involving children
important_types = pd.Series(['BATTERY', 'ROBBERY', 'THEFT', 'BURGLARY',
       'ASSAULT', 'CRIM SEXUAL ASSAULT', 'KIDNAPPING', 'SEX OFFENSE', 
                             'HOMICIDE', 'INTIMIDATION'])

In [7]:
#Removes any locations not adjacent to a street or path
important_location = pd.Series(['STREET', 'SIDEWALK',
       'PARKING LOT/GARAGE(NON.RESID.)',
       'DRIVEWAY - RESIDENTIAL', 'ALLEY', 'CTA BUS STOP',
       'VACANT LOT/LAND',
       'RESIDENTIAL YARD (FRONT/BACK)', 
       'COLLEGE/UNIVERSITY GROUNDS', 
       'PARK PROPERTY', 
       'CHA PARKING LOT/GROUNDS', 
       'LAKEFRONT/WATERFRONT/RIVERBANK',
       'BRIDGE', 'PARKING LOT', 
       'CEMETARY', 'HIGHWAY/EXPRESSWAY', 'YARD', 
       'VACANT LOT', 'CHA PARKING LOT', 
       'DRIVEWAY', 
       'CHA GROUNDS', 'RIVER', 'LAGOON',
       'DUMPSTER', 'WOODED AREA', 'RIVER BANK'])

In [8]:
#Keep only relevent columns 
important_cols = ['Case Number', 'Date', 'Block', 'IUCR', 'Primary Type',
       'Description', 'Location Description', 'Arrest', 'X Coordinate',
       'Y Coordinate', 'Latitude', 'Longitude']

In [10]:
DF1 = DF[(DF['Primary Type'].isin(important_types))&(
    DF['Location Description'].isin(important_location))]
DF1 = DF1[important_cols]

In [12]:
#Standardize block names
DF1.Block = DF1.Block.str.upper()

In [14]:
#Estimate Coordinates for any data where block name is given but coordinates are not
DF1["X Coordinate"] = DF1.groupby("Block")[
    "X Coordinate"].transform(lambda x: x.fillna(x.mean()))
DF1["Y Coordinate"] = DF1.groupby("Block")[
    "X Coordinate"].transform(lambda x: x.fillna(x.mean()))
DF1["Latitude"] = DF1.groupby("Block")[
    "Latitude"].transform(lambda x: x.fillna(x.mean()))
DF1["Longitude"] = DF1.groupby("Block")[
    "Longitude"].transform(lambda x: x.fillna(x.mean()))

In [17]:
#Remove any data for which coordinates cannot be obtained
DF1 = DF1.dropna()

In [16]:
#Convert date axis
DF1['Date'] = pd.to_datetime(DF1['Date'])

In [18]:
#Create new day/hour columns 
DF1['Day'] = DF1['Date'].dt.date
DF1['Hour'] = DF1['Date'].dt.hour

In [20]:
#Fix columns so they can be used in SQL 
cols = [k.replace(' ', '_') for k in DF1.columns]
DF1.columns = cols

In [21]:
#Write to SQL
DF1.to_sql('CrimeData1', c, if_exists = 'replace', index = True)

In [3]:
#Data received via request from https://www.ncdc.noaa.gov/cdo-web/
Weather = pd.read_csv('DailyWeather.csv')

In [8]:
#Keep only relevent columns 
Weather = Weather[['DATE', 'DAILYAverageDryBulbTemp', 'DAILYPrecip']]
Weather.columns = ['Date', 'AverageTemp', 'Precip']

In [ ]:
#Clean data
Weather['Precip'] = Weather.Precip.str.replace('T','0.001')
Weather['Precip'] = Weather.Precip.str.replace('s','')
Weather['Precip'] = Weather.Precip.astype('float64')

In [25]:
#Convert Date values 
Weather['Date'] = pd.to_datetime(Weather['Date'])
Weather['Date'] = Weather.Date.dt.date

In [33]:
Send to SQL
Weather.to_sql('DailyWeather', c, if_exists = 'replace', index = False)

In [23]:
#Close SQL connection 
c.close()